# Installs

In [ ]:
!pip install tensorflow==2.8.1

In [11]:
pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.5
    Uninstalling protobuf-4.25.5:
      Successfully uninstalled protobuf-4.25.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.62.3 requires protobuf>=4.21.6, but you have protobuf 3.20.3 which is incompatible.
pandas-gbq 0.24.0 requires google-auth-oauthlib>=0.7.0, but you have google-auth-oauthlib 0.4.6 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.8.1 which is incompatible.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pennylane as qml
from pennylane import numpy as np
from tensorflow.keras.models import Model, Sequential

In [ ]:
!pip install pennylane

In [ ]:
# Install the Kaggle library
!pip install kaggle

# Upload kaggle.json file
from google.colab import files
files.upload()  # This will prompt you to upload kaggle.json

# Make a directory for Kaggle and move the file there
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Now you can download the dataset
!kaggle datasets download -d mlg-ulb/creditcardfraud -p /content/datasets --unzip

print("Dataset downloaded to: /content/datasets")

# Data Loading

In [25]:
finance_df=pd.read_csv("/content/datasets/creditcard.csv",delimiter=',')

NameError: name 'pd' is not defined

In [26]:
finance_df.describe()

NameError: name 'finance_df' is not defined

In [ ]:
df=finance_df

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

train_size = 0.7  # 70% for training
test_size = 0.15  # 15% for testing
validation_size = 0.15  # 15% for validation

# First split: Train and temp (temp is a placeholder for validation + test)
train_df, temp_df = train_test_split(df, test_size=(test_size + validation_size), random_state=42)

# Second split: Split the temp set into validation and test
validation_df, test_df = train_test_split(temp_df, test_size=test_size / (test_size + validation_size), random_state=42)

# Output the splits
print(f"Train set size: {len(train_df)}")
print(f"Validation set size: {len(validation_df)}")
print(f"Test set size: {len(test_df)}")

In [ ]:
y_train = train_df['Class']
x_train = train_df.drop(['Class'], axis=1)
y_validation = validation_df ['Class']
x_validation = validation_df.drop(['Class'], axis=1)
y_test = test_df['Class']
x_test = test_df.drop(['Class'], axis=1)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,StandardScaler
import tensorflow as tf

In [ ]:
lbl_clf = LabelEncoder()
y_train = lbl_clf.fit_transform(y_train)
y_train = tf.keras.utils.to_categorical(y_train)

In [ ]:
std_clf = StandardScaler()
x_train = std_clf.fit_transform(x_train)
x_validation = std_clf.fit_transform(x_validation)
x_test = std_clf.transform(x_test)

In [ ]:
y_train

# Model


Now we start initializing the Quantum Circuit

In the following code we can change the defauly_qubit to AWS services by using the following commands:

device_arn = "arn:aws:braket:eu-west-2::device/qpu/ionq/Aria-1"
dev = qml.device('braket.aws.qubit',device_arn=device_arn, wires=num_wires)

TypeError: Descriptors cannot be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.layers import Dense, Dropout

hidden = Dense(32, activation ="relu", kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l2(0.01))
out_2 = Dense(9, activation ="relu", kernel_initializer='he_normal', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001))
do = Dropout(0,3)

In [4]:
num_wires = 3
dev = qml.device('default.qubit', wires=num_wires)

In [17]:
@qml.qnode(dev, interface="tf", diff_method="backprop")
def quantum_nn(inputs, weights):
    # print(inputs)
    # print(inputs[2])
    qml.RY(inputs[0], wires=0)
    qml.RY(inputs[1], wires=1)
    qml.RY(inputs[2], wires=2)
    qml.Rot(weights[0] * inputs[3], weights[1] * inputs[4], weights[2] * inputs[5], wires=1)
    qml.Rot(weights[3] * inputs[6], weights[4] * inputs[7], weights[5] * inputs[8], wires=2)
    qml.CNOT(wires=[1, 2])
    qml.RY(weights[6], wires=2)
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[1, 2])
    # print("Now we are returning the evaluations: ")
    return [qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(2))]

## Understanding and testing the quantum circuit

In [6]:
# Prepare dummy input data and weights
inputs = tf.constant([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], dtype=tf.float64)
weights = tf.Variable([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7], dtype=tf.float64)

In [7]:
outputs = quantum_nn(inputs, weights)
probs = [tf.constant((output + 1) / 2) for output in outputs]
softmax_outputs = tf.nn.softmax(outputs)

print("Softmax Outputs:", softmax_outputs)
print("Normalized Outputs:", probs)
print("Untouched Outputs:", outputs)

tf.Tensor([0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9], shape=(9,), dtype=float64)
tf.Tensor(0.3, shape=(), dtype=float64)
Now we are returning the evaluations: 
Softmax Outputs: tf.Tensor([0.6718629 0.3281371], shape=(2,), dtype=float64)
Normalized Outputs: [<tf.Tensor: shape=(), dtype=float64, numpy=0.9975020826390127>, <tf.Tensor: shape=(), dtype=float64, numpy=0.6391906937035146>]
Untouched Outputs: [<tf.Tensor: shape=(), dtype=float64, numpy=0.9950041652780255>, <tf.Tensor: shape=(), dtype=float64, numpy=0.2783813874070292>]


Even though we implemented some reguralization techniques, for now we are going to use the outputs as they are given back from the circuit

In [8]:
threshold = 0.5
binary_probs = [tf.constant(1 if output > threshold else 0) for output in outputs]

print(binary_probs)

[<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=0>]


In [9]:
# def loss_fn(inputs, weights, targets):
#     # Compute outputs for the batch
#     outputs = [quantum_nn(input_sample, weights) for input_sample in inputs]  # Outputs: [batch_size, num_classes]

#     # Threshold to get binary probabilities
#     threshold = 0.5
#     binary_probs = [
#         [1.0 if output > threshold else 0.0 for output in record]
#         for record in outputs
#     ]

#     # Convert binary_probs to a proper tensor
#     binary_probs = tf.constant(binary_probs, dtype=tf.float64)

#     print("Binary Probabilities:", binary_probs)

#     # Map binary probabilities to class labels
#     predicted_labels = [1 if prob[1] == 1.0 else 0 for prob in binary_probs]
#     predicted_labels=tf.cast(predicted_labels,dtype=tf.float64)
#     print("Predicted Labels:", predicted_labels)
#     # Ensure targets are also tensors of the same dtype
#     targets = tf.cast(targets, dtype=tf.float64)

#     print(targets)

#     # Compute binary cross-entropy loss

#     loss = tf.keras.losses.binary_crossentropy(targets, predicted_labels)
#     return tf.reduce_mean(loss)


In [15]:
import warnings


def loss_fn(inputs, weights, targets):
    # Compute outputs for the batch
    outputs = [quantum_nn(input_sample, weights) for input_sample in inputs]  # Raw continuous outputs

    softmax_outputs = tf.nn.softmax(outputs)

    # Convert to tensor
    outputs = tf.convert_to_tensor(softmax_outputs, dtype=tf.float64)

    # print("Raw Outputs:", outputs)

    # Ensure targets are tensors of the same dtype
    targets = tf.cast(targets, dtype=tf.float64)

    # Compute binary cross-entropy loss directly on raw outputs
    warnings.filterwarnings("ignore")
    loss = tf.keras.losses.binary_crossentropy(targets, outputs)
    return tf.reduce_mean(loss)


In [27]:
import numpy as np
import tensorflow as tf

# Generate 150 samples of 8-dimensional inputs
inputs = tf.constant(np.random.rand(1500, 9), dtype=tf.float64)

# Generate 150 one-hot encoded targets (binary classification)
binary_labels = np.random.choice([0, 1], size=(1500,))
targets = tf.constant([[1, 0] if label == 0 else [0, 1] for label in binary_labels], dtype=tf.float64)

print("Inputs shape:", inputs.shape)
print("Targets shape:", targets.shape)

Inputs shape: (1500, 9)
Targets shape: (1500, 2)


In [28]:
weights = tf.Variable([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8], dtype=tf.float64)


# Forward pass
loss_value = loss_fn(inputs, weights, targets)

print("Loss Value:", loss_value.numpy())

Loss Value: 0.7696314555729998


In [22]:
# Compute gradients
with tf.GradientTape() as tape:
    tape.watch(weights)  # Ensure weights are tracked
    loss_value = loss_fn(inputs, weights, targets)

# Compute gradient of loss with respect to weights
grads = tape.gradient(loss_value, weights)

print("Loss:", loss_value.numpy())
print("Gradients:", grads.numpy())

Loss: 0.8010057081914876
Gradients: [ 5.52254811e-05 -1.89640936e-02 -1.26185200e-18 -1.65395865e-02
  9.50913059e-02 -1.87721798e-02  1.51683186e-01  0.00000000e+00]


In [23]:
# Set up the optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [29]:
# Number of training epochs
epochs = 10  # Adjust as needed

# Training loop
for epoch in range(epochs):
    with tf.GradientTape() as tape:
        loss = loss_fn(inputs, weights, targets)
    grads = tape.gradient(loss, weights)
    optimizer.apply_gradients(zip([grads], [weights]))
    print(f"Epoch {epoch + 1}, Loss: {loss.numpy()}")


Epoch 1, Loss: 0.7696314555729998
Epoch 2, Loss: 0.7684706800371814
Epoch 3, Loss: 0.7669225819735259
Epoch 4, Loss: 0.7651355173740946
Epoch 5, Loss: 0.7631910215346874
Epoch 6, Loss: 0.7611427433521127
Epoch 7, Loss: 0.7590291133218525
Epoch 8, Loss: 0.7568790305100238
Epoch 9, Loss: 0.7547148906430472
Epoch 10, Loss: 0.7525543822112587


## Now implementing hybrid model

In [ ]:
def hybrid_model(num_layers, num_wires):
    weight_shapes = {"weights": (7,)}
    print(weight_shapes)
    qlayer = qml.qnn.KerasLayer(quantum_nn, weight_shapes, output_dim=2)
    hybrid_model = tf.keras.Sequential([hidden,do, out_2,do,qlayer])
    return hybrid_model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential
import pennylane as qml
import numpy as np

# Define the device for the quantum layer
dev = qml.device("default.qubit", wires=3)

# Correct the Dropout rate from 0,3 to 0.3
do = Dropout(0.3)

# Define the quantum neural network
@qml.qnode(dev, interface="tf", diff_method="backprop")
def quantum_nn(inputs, weights):
    print(inputs[0])
    # Encode inputs into quantum states
    qml.RY(inputs[0], wires=0)
    qml.RY(inputs[1], wires=1)
    qml.RY(inputs[2], wires=2)

    # Parameterized quantum gates
    qml.Rot(weights[0] * inputs[3], weights[1] * inputs[4], weights[2] * inputs[5], wires=1)
    qml.Rot(weights[3] * inputs[6], weights[4] * inputs[7], weights[5] * inputs[8], wires=2)

    # Entangling gates
    qml.CNOT(wires=[1, 2])
    qml.RY(weights[6], wires=2)
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[1, 2])

    print("built")

    # Measure expectation values
    return [qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(2))]

# Build the hybrid model
def hybrid_model():
    # Define weight shapes for the quantum layer
    weight_shapes = {"weights": (7,)}

    # Define the quantum Keras layer
    qlayer = qml.qnn.KerasLayer(quantum_nn, weight_shapes, output_dim=2)

    # Define the classical layers
    hidden = Dense(
        32,
        activation="relu",
        kernel_initializer='he_normal',
        kernel_regularizer=tf.keras.regularizers.l2(0.01)
    )
    out_2 = Dense(
        9,
        activation="relu",
        kernel_initializer='he_normal',
        kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)
    )


    out_3 = Dense(
        2,
        activation="relu",
        kernel_initializer='he_normal',
        kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)
    )

    # Construct the sequential model
    model = Sequential([hidden, do, out_2, do,qlayer])
    return model

# Create dummy X_train data with 15 features
num_samples = 90  # For demonstration purposes
X_train = np.random.rand(num_samples, 15).astype('float32')

# Build and compile the model
model = hybrid_model()
model.compile(optimizer='adam', loss='mse')

# print(output[0])

In [ ]:
X_train

array([[0.33416817, 0.4570113 , 0.7913605 , ..., 0.67572916, 0.8719203 ,
        0.3107077 ],
       [0.16494113, 0.50823504, 0.46302342, ..., 0.25933972, 0.37235272,
        0.13594991],
       [0.5896279 , 0.2759565 , 0.24129827, ..., 0.7974579 , 0.29604778,
        0.5593871 ],
       ...,
       [0.2396103 , 0.41682413, 0.3932262 , ..., 0.81381345, 0.8443339 ,
        0.4086366 ],
       [0.17334467, 0.7532806 , 0.37795654, ..., 0.41479996, 0.9777221 ,
        0.4702656 ],
       [0.70583165, 0.28386775, 0.9448934 , ..., 0.12347304, 0.7984639 ,
        0.3674435 ]], dtype=float32)

In [ ]:
outputs = model(X_train)

# Print the outputs
print("Model outputs:")
print(outputs)

tf.Tensor(
[0.         0.         0.         0.36025834 0.         0.17672184
 0.         0.         0.        ], shape=(9,), dtype=float32)
built


InvalidArgumentError: Exception encountered when calling layer 'keras_layer_3' (type KerasLayer).

{{function_node __wrapped__Reshape_device_/job:localhost/replica:0/task:0/device:CPU:0}} Input to reshape is a tensor with 9 values, but the requested shape has 90 [Op:Reshape]

Call arguments received by layer 'keras_layer_3' (type KerasLayer):
  • inputs=tf.Tensor(shape=(90, 9), dtype=float32)

In [ ]:
print(outputs[0][0])

# Now exactly what the delloite site say:

# Train/Test

In [ ]:
import tensorflow as tf

# Example: Define a custom metric
def custom_metric(y_true, y_pred):
    return tf.reduce_mean(tf.abs(y_true - y_pred))  # Mean Absolute Error (MAE)

In [ ]:
hybrid_q_model = hybrid_model()
hybrid_q_model.compile(optimizer='adam', loss = 'MSE', metrics = [custom_metric])

In [ ]:
y_train_list = np.array(y_train).flatten().tolist()

class_counts = np.bincount(y_train_list)
class_frequencies = class_counts / float(len(y_train))
class_weights = 1 / np.sqrt(class_frequencies)-0.2



In [ ]:
class_weight_dict = {i: weight for i, weight in enumerate(class_weights)}

In [ ]:
class_weight_dict

In [ ]:
input_dim = x_train.shape[1]  # Number of columns in x_train
hybrid_q_model.build(input_shape=(None, 30))  # Replace `input_dim` with the feature size
hybrid_q_model.summary()

In [ ]:
history = hybrid_q_model.fit(x_train, y_train, epochs = 3, batch_size = 200, validation_data=(x_validation, y_validation),shuffle=True)

# TRY 2

In [ ]:
fraud_id = finance_df.Class[finance_df.Class.eq(1)].index
nofraud_id = finance_df.Class[finance_df.Class.eq(0)].sample(len(fraud_id)*3).index

df_reduced = finance_df.loc[fraud_id.union(nofraud_id)].sample(frac = 1).reset_index( drop = True);

In [ ]:
print(fraud_id.shape, nofraud_id.shape)

(492,) (1476,)


In [ ]:
df_reduced.shape

(1968, 31)

In [ ]:
finance_df=df_reduced

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import tensorflow as tf

# Define the quantum device
dev = qml.device("default.qubit", wires=3)

# Define the quantum neural network
@qml.qnode(dev, interface="tf", diff_method="backprop")
def quantum_nn(inputs, weights):
    print(f"Inputs received: {inputs}")
    print(f"Weights received: {weights}")

    # Encode inputs into quantum states
    qml.RY(inputs[0], wires=0)
    qml.RY(inputs[1], wires=1)
    qml.RY(inputs[2], wires=2)
    print("Encoded inputs into quantum states.")

    # Parameterized quantum gates
    qml.Rot(weights[0] * inputs[3], weights[1] * inputs[4], weights[2] * inputs[5], wires=1)
    qml.Rot(weights[3] * inputs[6], weights[4] * inputs[7], weights[5] * inputs[8], wires=2)
    print("Applied parameterized quantum gates.")

    # Entangling gates
    qml.CNOT(wires=[1, 2])
    qml.RY(weights[6], wires=2)
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[1, 2])
    print("Added entangling gates.")

    # Measure expectation values
    expvals = [qml.expval(qml.PauliZ(0)), qml.expval(qml.PauliZ(2))]
    print(f"Measured expectation values: {expvals}")
    return expvals

In [ ]:
def print_circuit(inputs, weights):
    print("Printing the quantum circuit...")
    circuit_drawer = qml.draw(quantum_nn)
    print(circuit_drawer(inputs, weights))

In [ ]:
# Example inputs and weights
inputs = np.random.random(9)  # Replace with actual inputs
weights = np.random.random(7)  # Replace with actual weights

# Define the model using the functional API
input_layer = tf.keras.layers.Input(shape=(9,))

dense_layer = tf.keras.layers.Dense(7, activation='relu')(input_layer)

# Apply the quantum neural network via a Lambda layer
quantum_layer = tf.keras.layers.Lambda(lambda x: tf.stack(quantum_nn(x, weights), axis=-1))(dense_layer)

# Output layer: We need to concatenate the two outputs into a single tensor
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(quantum_layer)

# Build the model using the functional API
model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Debug prints
print("Model created.")
print_circuit(inputs, weights)

Inputs received: Tensor("Placeholder:0", shape=(None, 7), dtype=float32)
Weights received: [0.19321466 0.5180692  0.27689942 0.73907562 0.51439225 0.05222705
 0.41112046]
Encoded inputs into quantum states.
Applied parameterized quantum gates.
Added entangling gates.
Measured expectation values: [expval(Z(0)), expval(Z(2))]
Model created.
Printing the quantum circuit...
Inputs received: [0.48457208 0.48337721 0.57236355 0.29227794 0.29503232 0.33084619
 0.69705429 0.72817349 0.08799096]
Weights received: [0.19321466 0.5180692  0.27689942 0.73907562 0.51439225 0.05222705
 0.41112046]
Encoded inputs into quantum states.
Applied parameterized quantum gates.
Added entangling gates.
Measured expectation values: [expval(Z(0)), expval(Z(2))]
0: ──RY(0.48)───────────────────────────────────╭●────┤  <Z>
1: ──RY(0.48)──Rot(0.06,0.15,0.09)─╭●───────────│──╭●─┤     
2: ──RY(0.57)──Rot(0.52,0.37,0.00)─╰X──RY(0.41)─╰X─╰X─┤  <Z>


In [ ]:
# Generate example input data (batch of size 1 with 9 features)
example_input = np.random.random((1, 9))  # Replace with actual data as needed

# Pass the data through the model
output = model.predict(example_input)

# Print the input and corresponding output
print(f"Input: {example_input}")
print(f"Output: {output}")


Inputs received: Tensor("model_9/dense_31/Relu:0", shape=(None, 7), dtype=float32)
Weights received: [0.19321466 0.5180692  0.27689942 0.73907562 0.51439225 0.05222705
 0.41112046]
Encoded inputs into quantum states.
Applied parameterized quantum gates.
Added entangling gates.
Measured expectation values: [expval(Z(0)), expval(Z(2))]


InvalidArgumentError: Graph execution error:

Detected at node 'model_9/lambda_12/strided_slice_1' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>
      ColabKernelApp.launch_instance()
    File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever
      self._run_once()
    File "/usr/lib/p

Inputs received: Tensor("Placeholder:0", shape=(None, 9), dtype=float32)
Weights received: [0.92048626 0.88384354 0.74917156 0.44352705 0.4254672  0.58196584
 0.97585557]


TypeError: Exception encountered when calling layer "lambda_9" (type Lambda).

'NoneType' object cannot be interpreted as an integer

Call arguments received by layer "lambda_9" (type Lambda):
  • inputs=tf.Tensor(shape=(None, 9), dtype=float32)
  • mask=None
  • training=None